In [4]:
# Header block to include all modules that must be imported ahead of time
# Only needs to be run once per session, and each time a new module is added

import requests
import json
import pandas as pd # this will need to be installed via command line first

In [60]:
# Method to get all the revelent player attributes for every player currently signed on every NFL team
# The CBS Sports website is used as the source

def GetPlayterAttributes(team: list): # accepts a list ['team-name', 'LOC']
    # Use requests to get the raw HTML response from cbssports.com
    CBSResponse = requests.get(f'https://www.cbssports.com/nfl/teams/{team[1]}/{team[0]}/roster/')
    
    # If we get a normal response proceed with data scraping
    if CBSResponse.status_code == 200:
                 
        # Confirm the request was successful
        print(f"Request succeeded with status code {CBSResponse.status_code}")
        
        # Use Pandas to read the HTML content and put it into a DataFrame
        TableTag = 'TableBase-table'
        DataFrames = pd.read_html(CBSResponse.content, attrs={'class': TableTag})
        
        # Select the relevant data from the DataFrame(offense, defense, and special teams)
        OffenseDF = DataFrames[0][['Player', 'POS', 'HT', 'WT', 'DOB (AGE)', 'EXP']]
        DefenseDF = DataFrames[1][['Player', 'POS', 'HT', 'WT', 'DOB (AGE)', 'EXP']]
        SpecialDF = DataFrames[2][['Player', 'POS', 'HT', 'WT', 'DOB (AGE)', 'EXP']]
        
        # Combine the three DataFrames into one
        FullTeamDF = pd.concat([OffenseDF, DefenseDF, SpecialDF], axis = 0, ignore_index = True)
        print(FullTeamDF.head(20))
        
        # Method to clean the player names since CBS puts player news directly in the name cell
        def CleanNameCell(NameText):
            # Split the player name into initial, first-last name string, and last name string
            NameText = NameText.split(' ')
            
            # If the name cell doesn't contain player news and no name suffix
            if len(NameText) == 3:
                # Remove the first initial
                NameText = NameText[1:]
                # Extract the first name from the first-last name string
                NameText[0] = NameText[0][len(NameText[1]):]
                # Create new string with first and last name
                NameText = ' '.join(NameText)
                
            # The name cell contains a name suffix
            elif len(NameText) == 5:
                # Extract the first name from the first-suffix name string
                NameText[0] = NameText[2][len(NameText[4]):]
                # Change the 3rd element to the suffix
                NameText[2] = NameText[4]
                # Turn the list into a string
                NameText = ' '.join(NameText[:3])

            # Otherwise the player name cell contains player news
            else:
                # Select 3rd, 4th elements from NameText since they contain player's first, last name
                NameTextCapitalized = [s for s in NameText if s and not s[0].islower()]
                print(NameTextCapitalized)
                """
                NameText[0] = NameText[4] 
                NameText[1] = NameText[5][:-1]
                # print(NameText)
                NameText = ' '.join(NameText[:3])
                print(NameText)
                """
            return NameText
        
        # Apply the method to the player names column
        FullTeamDF['Player'] = FullTeamDF['Player'].apply(CleanNameCell)
        print(FullTeamDF.head(20))
               
        """
        # Create a .csv file with all the player names and their cap hits
        SalariesDF.to_csv(f'{TeamNameCaptalized}-Player-Salary-List.csv', index = False, header = False)
        
        # Read the csv back for testing purposes
        SalariesCSV = pd.read_csv(f'{TeamNameCaptalized}-Player-Salary-List.csv', header = None)
        SalariesCSV.columns = ['Player', 'CapNumber', 'TeamLOC']
        print(SalariesCSV)
        """

    # If we don't get a normal reponse, stop scraping
    else:
        print(f"Request failed with status code {CBSResponse.status_code}")
        return -1
    
GetPlayterAttributes(['seattle-seahawks', 'SEA'])

Request succeeded with status code 200
                                               Player POS    HT   WT   
0                                 S. HowellSam Howell  QB   6-1  220  \
1                              S. MannionSean Mannion  QB   6-6  230   
2                                  G. SmithGeno Smith  QB   6-3  221   
3                        Z. CharbonnetZach Charbonnet  RB   6-1  214   
4                              G. HolaniGeorge Holani  RB  5-11  210   
5                                  K. LewisKobe Lewis  RB   5-9  208   
6                           K. McIntoshKenny McIntosh  RB   6-0  204   
7   R. Person Jr.  Seahawks' Ricky Person: Joins S...  RB   6-0  217   
8                     K. Walker IIIKenneth Walker III  RB   5-9  211   
9                                    J. BoboJake Bobo  WR   6-4  207   
10  D. Eskridge  Seahawks' Dee Eskridge: Taking pa...  WR   5-9  190   
11                             H. HattenHayden Hatten  WR   6-1  207   
12                       

In [3]:
# Method to get the revelent attributes for every player currently signed on every NFL team
# Reads in the team list csv, and calls the GetTeamSalaryCapHits method for each team
def GetAllPlayerAttributes():
    # Read in the full team list csv
    FullTeamListCSV = pd.read_csv('./data/team-attribute-lists/Teams-List.csv', header = None)
    FullTeamListCSV.columns = ['TeamName', 'Abbreviation', 'Conference', 'Division']
    
    # Create a list of all the team names and abbreviations as a list of lists
    TeamList = []
    # Take the first two columns of each row, convert to a list, append to TeamList
    FullTeamListCSV.apply(lambda row: TeamList.append(row[:2].tolist()), axis = 1)
    
    # Get the salary cap hits for each team
    for team in TeamList:
        # format the team name to be used in the URL
        team[0] = team[0].replace(" ", "-").lower()
        GetTeamSalary(team)
    

GetAllTeamSalary()
"""
# A list of links to Over The Cap Salary Cap pages for various teams
NFLTeams = [['buffalo-bills', 'BUF', 4], ['miami-dolphins', 'MIA', 20], ['new-england-patriots', 'NE0', 22], ['new-york-jets', 'NYJ', 25],
            ['baltimore-ravens', 'BAL', 3], ['cincinnati-bengals', 'CIN', 7], ['cleveland-browns', 'CLE', 8], ['pittsburgh-steelers', 'PIT', 27],
            ['houston-texans', 'HOU', 13], ['indianapolis-colts', 'IND', 14], ['jacksonville-jaguars', 'JAX', 15], ['tennessee-titans', 'TEN', 31], 
            ['denver-broncos', 'DEN', 10], ['kansas-city-chiefs', 'KC0', 16], ['las-vegas-raiders', 'LV0', 17], ['los-angeles-chargers', 'LAC', 18],
            ['dallas-cowboys', 'DAL', 9], ['new-york-giants', 'NYG', 24], ['philadelphia-eagles', 'PHI', 26], ['washington-commanders', 'WSH', 32],
            ['chicago-bears', 'CHI', 6], ['detroit-lions', 'DET', 11], ['green-bay-packers', 'GB0', 12], ['minnesota-vikings', 'MIN', 21],
            ['atlanta-falcons', 'ATL', 2], ['carolina-panthers', 'CAR', 5], ['new-orleans-saints', 'NO0', 23], ['tampa-bay-buccaneers', 'TB0', 30],
            ['arizona-cardinals', 'ARI', 1], ['los-angeles-rams', 'LAR', 19], ['san-francisco-49ers', 'SF0', 28], ['seattle-seahawks', 'SEA', 29]
           ]

GetTeamRoster(['buffalo-bills', 'BUF', 4])
"""

buffalo-bills BUF 4


AttributeError: 'str' object has no attribute 'capabilities'